### This Notebook demonstrates the simple use of cross validation to evaluate a model
function used:
1. cross_val_score
2. gridsearchcv

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error

/var/folders/08/0mmxpjds48s_7kghtt0p4y9c0000gn/T/ipykernel_45467/1850849659.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Load the data from the UCI repository
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data', 
                 header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [4]:
# Add column names
df.columns = ['age', 
              'sex', 
              'cp', 
              'restbp', 
              'chol', 
              'fbs', 
              'restecg', 
              'thalach', 
              'exang', 
              'oldpeak', 
              'slope', 
              'ca', 
              'thal', 
              'hd']
df.head()

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [5]:
# look at the data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   age      303 non-null    float64
 1   sex      303 non-null    float64
 2   cp       303 non-null    float64
 3   restbp   303 non-null    float64
 4   chol     303 non-null    float64
 5   fbs      303 non-null    float64
 6   restecg  303 non-null    float64
 7   thalach  303 non-null    float64
 8   exang    303 non-null    float64
 9   oldpeak  303 non-null    float64
 10  slope    303 non-null    float64
 11  ca       303 non-null    object 
 12  thal     303 non-null    object 
 13  hd       303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB


In [6]:
# let's see the unique values in the 'ca' and 'thal' columns as they are objects column
print(df['ca'].unique())
print(df['thal'].unique())

['0.0' '3.0' '2.0' '1.0' '?']
['6.0' '3.0' '7.0' '?']


In [7]:
df.loc[(df['ca'] == '?' ) | (df['thal'] == '?')]

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
87,53.0,0.0,3.0,128.0,216.0,0.0,2.0,115.0,0.0,0.0,1.0,0.0,?,0
166,52.0,1.0,3.0,138.0,223.0,0.0,0.0,169.0,0.0,0.0,1.0,?,3.0,0
192,43.0,1.0,4.0,132.0,247.0,1.0,2.0,143.0,1.0,0.1,2.0,?,7.0,1
266,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,?,2
287,58.0,1.0,2.0,125.0,220.0,0.0,0.0,144.0,0.0,0.4,2.0,?,7.0,0
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,?,3.0,0


In [8]:
# as we can see, there are 6 rows with missing values, we can drop them
df = df.loc[(df['ca'] != '?' ) & (df['thal'] != '?')]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297 entries, 0 to 301
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   age      297 non-null    float64
 1   sex      297 non-null    float64
 2   cp       297 non-null    float64
 3   restbp   297 non-null    float64
 4   chol     297 non-null    float64
 5   fbs      297 non-null    float64
 6   restecg  297 non-null    float64
 7   thalach  297 non-null    float64
 8   exang    297 non-null    float64
 9   oldpeak  297 non-null    float64
 10  slope    297 non-null    float64
 11  ca       297 non-null    object 
 12  thal     297 non-null    object 
 13  hd       297 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 34.8+ KB


According to the website, the features description is:
- **age**, **Float**
- **sex** - **Category**
  - 0 = female
  - 1 = male
- **cp**, chest pain, **Category**
  - 1 = typical angina
  - 2 = atypical angina
  - 3 = non-anginal pain
  - 4 = asymptomatic
- **restbp**, resting blood pressure (in mm Hg), **Float**
- **chol**, serum cholesterol in mg/dl, **Float**
- **fbs**, fasting blood sugar, **Category**
  - 0 = >=120 mg/dl
  - 1 = <120 mg/dl
- **restecg**, resting electrocardiographic results, **Category**
  - 1 = normal
  - 2 = having ST-T wave abnormality
  - 3 = showing probable or definite left ventricular hypertrophy
- **thalach**,  maximum heart rate achieved, **Float**
- **exang**, exercise induced angina, **Category**
  - 0 = no
  - 1 = yes
- **oldpeak**, ST depression induced by exercise relative to rest. **Float**
- **slope**, the slope of the peak exercise ST segment, **Category**
  - 1 = upsloping
  - 2 = flat
  - 3 = downsloping
- **ca**, number of major vessels (0-3) colored by fluoroscopy, **Float**
- **thal**, thalium heart scan, **Category**
  - 3 = normal (no cold spots)
  - 6 = fixed defect (cold spots during rest and exercise)
  - 7 = reversible defect (when cold spots only appear during exercise)

In [10]:
# according to the descriptions, the columns cp,  restecg, slope, and thal are categorical variables. However they are stored as float. 
# We need to convert them to int type. And then we should covert them use one-hot encoding

# convert the columns to int type
df['cp'] = df['cp'].astype(int)
df['fbs'] = df['fbs'].astype(int)
df['restecg'] = df['restecg'].astype(int)
df['slope'] = df['slope'].astype(int)
df['thal'] = pd.to_numeric(df['thal']).astype(int)
df['exang'] = df['exang'].astype(int)
df['ca'] = pd.to_numeric(df['ca'])

In [11]:
X = df.drop('hd', axis=1).copy()
y = df['hd'].copy()

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# create a column transformer for one-hot encoding
categorical_columns = ['cp', 'restecg', 'slope', 'thal']
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_columns)], 
                       remainder='passthrough')
X_encoder = ct.fit_transform(X)

In [13]:
ct.get_feature_names_out()

array(['encoder__cp_1', 'encoder__cp_2', 'encoder__cp_3', 'encoder__cp_4',
       'encoder__restecg_0', 'encoder__restecg_1', 'encoder__restecg_2',
       'encoder__slope_1', 'encoder__slope_2', 'encoder__slope_3',
       'encoder__thal_3', 'encoder__thal_6', 'encoder__thal_7',
       'remainder__age', 'remainder__sex', 'remainder__restbp',
       'remainder__chol', 'remainder__fbs', 'remainder__thalach',
       'remainder__exang', 'remainder__oldpeak', 'remainder__ca'],
      dtype=object)

In [14]:
X_encoder.shape, y.shape

((297, 22), (297,))

In [15]:
y.unique()

array([0, 2, 1, 3, 4])

In [16]:
# as the target is to predict if the patient has heart disease or not, we can convert the target to binary

y.replace([1, 2, 3, 4], 1, inplace=True)

In [17]:
y.unique()

array([0, 1])

In [18]:
# build decision tree model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X_encoder, y, test_size=0.2, random_state=42)

In [29]:
dc = DecisionTreeClassifier(random_state=42)
dc.fit(X_train, y_train)

y_pred = dc.predict(X_test)
roc_auc_score(y_test, y_pred)

0.8402777777777778

In [20]:
# now we can use cross validation to fine tune the model and see if we can improve the performance
# lets try to use different alpha values to see if we can improve the performance

alpha_values = [0.0001, 0.001, 0.01, 0.1, 0.015, 0.016, 0.017, 0.014]

for alpha in alpha_values:
    dc = DecisionTreeClassifier(random_state=42, ccp_alpha=alpha)
    dc.fit(X_train, y_train)
    y_pred = dc.predict(X_test)
    print(f'alpha: {alpha}, roc_auc: {roc_auc_score(y_test, y_pred)}')

alpha: 0.0001, roc_auc: 0.8402777777777778
alpha: 0.001, roc_auc: 0.8402777777777778
alpha: 0.01, roc_auc: 0.8472222222222223
alpha: 0.1, roc_auc: 0.6736111111111112
alpha: 0.015, roc_auc: 0.8055555555555556
alpha: 0.016, roc_auc: 0.8472222222222222
alpha: 0.017, roc_auc: 0.8472222222222222
alpha: 0.014, roc_auc: 0.8055555555555556


In [21]:
# from the above we can see that when alpah is 0.015, the roc_auc score is the highest
# now lets see what the accuracy looks like on cross validation when alpha is 0.015

dc = DecisionTreeClassifier(random_state=42, ccp_alpha=0.015)
scores = cross_val_score(dc, X_train, y_train, cv=5, scoring='roc_auc')

print(f'roc_auc scores: {scores}')

roc_auc scores: [0.73913043 0.80608696 0.80818182 0.66181818 0.87137681]


In [22]:
# from the above we can see that the roc_auc scores are not consistent on different folds, this means that the data split is sensitive to the model performance
# we can try use cross validation with different alpha values to see if we can get a better performance

alpha_values = [0.0001, 0.001, 0.01, 0.1, 0.015, 0.016, 0.017, 0.014]
for alpha in alpha_values:
    dc = DecisionTreeClassifier(random_state=42, ccp_alpha=alpha)
    scores = cross_val_score(dc, X_train, y_train, cv=5, scoring='roc_auc')
    # average score for each alpha on each 5 folds
    avg_score = np.mean(scores)
    print(f'alpha: {alpha}, roc_auc scores: {avg_score}')

alpha: 0.0001, roc_auc scores: 0.7422575757575758
alpha: 0.001, roc_auc scores: 0.7422575757575758
alpha: 0.01, roc_auc scores: 0.7671403162055336
alpha: 0.1, roc_auc scores: 0.7112997364953886
alpha: 0.015, roc_auc scores: 0.7773188405797102
alpha: 0.016, roc_auc scores: 0.7776824769433466
alpha: 0.017, roc_auc scores: 0.7765915678524375
alpha: 0.014, roc_auc scores: 0.7795006587615285


From above we can see that when alpha is 0.014, the average roc_auc score is the highest


In [26]:
# we can also use grid search to find the best alpha value to achieve the same result

from sklearn.model_selection import GridSearchCV
param_grid = {'ccp_alpha': alpha_values}
dc = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dc, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

grid_search.best_score_, grid_search.best_params_

(0.7795006587615285, {'ccp_alpha': 0.014})

In [30]:
# retrain the model using the best alpha value

dc = DecisionTreeClassifier(random_state=42, ccp_alpha=0.014)
dc.fit(X_train, y_train)
y_pred = dc.predict(X_test)

roc_auc_score(y_test, y_pred)

0.8055555555555556